# Clustering Crypto

In [1]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from path import Path
import numpy as np
from IPython.display import display

# Amazon SageMaker and related imports
import sagemaker
import sagemaker.amazon.common as smac

!pip install -U altair
import altair as alt
%matplotlib inline

### Fetching Cryptocurrency Data

In [2]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path)
crypto_df.head()

Unnamed: 0   CoinName Algorithm  IsTrading ProofType  TotalCoinsMined  \
0         42    42 Coin    Scrypt       True   PoW/PoS     4.199995e+01   
1        365    365Coin       X11       True   PoW/PoS              NaN   
2        404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09   
3        611  SixEleven   SHA-256       True       PoW              NaN   
4        808        808   SHA-256       True   PoW/PoS     0.000000e+00   

  TotalCoinSupply  
0              42  
1      2300000000  
2       532000000  
3          611000  
4               0

### Data Preprocessing

In [3]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df.set_index("Unnamed: 0", inplace = True)
crypto_df.index.name = None
crypto_df.head()

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt       True   PoW/PoS     4.199995e+01              42
365    365Coin       X11       True   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       True       PoW              NaN          611000
808        808   SHA-256       True   PoW/PoS     0.000000e+00               0

In [4]:
# Keep only cryptocurrencies that are trading
crypto_trading_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_trading_df.head()

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt       True   PoW/PoS     4.199995e+01              42
365    365Coin       X11       True   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       True       PoW              NaN          611000
808        808   SHA-256       True   PoW/PoS     0.000000e+00               0

In [5]:
# Keep only cryptocurrencies with a working algorithm
crypto_trading_df = crypto_trading_df.dropna(axis=0, subset=['Algorithm'])
crypto_trading_df.head()

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt       True   PoW/PoS     4.199995e+01              42
365    365Coin       X11       True   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       True       PoW              NaN          611000
808        808   SHA-256       True   PoW/PoS     0.000000e+00               0

In [6]:
# Remove the 'IsTrading' column
crypto_trading_df = crypto_trading_df.drop(["IsTrading"], axis = 1)
crypto_trading_df.head()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
365    365Coin       X11   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       PoW              NaN          611000
808        808   SHA-256   PoW/PoS     0.000000e+00               0

In [7]:
# Remove rows with at least 1 null value
crypto_trading_df = crypto_trading_df.dropna(how='any',axis=0)
crypto_trading_df.head()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42      42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
404     404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
808         808   SHA-256   PoW/PoS     0.000000e+00               0
1337  EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
BTC     Bitcoin   SHA-256       PoW     1.792718e+07        21000000

In [8]:
# Remove rows with cryptocurrencies having no coins mined
crypto_trading_df = crypto_trading_df[crypto_trading_df['TotalCoinsMined'] != 0]
crypto_trading_df.head()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42      42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
404     404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
1337  EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
BTC     Bitcoin   SHA-256       PoW     1.792718e+07        21000000
ETH    Ethereum    Ethash       PoW     1.076842e+08               0

In [9]:
# Drop rows where there are 'N/A' text values
to_drop = ['N/A']
for column in crypto_trading_df.columns:
        crypto_trade_df = crypto_trading_df[~crypto_trading_df[column].isin(to_drop)]

crypto_trade_df.head()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42      42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
404     404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
1337  EliteCoin       X13   PoW/PoS     2.927942e+10    314159265359
BTC     Bitcoin   SHA-256       PoW     1.792718e+07        21000000
ETH    Ethereum    Ethash       PoW     1.076842e+08               0

In [10]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coins_name = crypto_trade_df[["CoinName"]]
coins_name.head()

CoinName
42      42 Coin
404     404Coin
1337  EliteCoin
BTC     Bitcoin
ETH    Ethereum

In [11]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_features_df = crypto_trade_df.drop(["CoinName"], axis = 1)
crypto_features_df.head()


Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42      Scrypt   PoW/PoS     4.199995e+01              42
404     Scrypt   PoW/PoS     1.055185e+09       532000000
1337       X13   PoW/PoS     2.927942e+10    314159265359
BTC    SHA-256       PoW     1.792718e+07        21000000
ETH     Ethash       PoW     1.076842e+08               0

In [12]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_features_df, columns=["Algorithm", "ProofType"])
X.head()

TotalCoinsMined TotalCoinSupply  Algorithm_1GB AES Pattern Search  \
42       4.199995e+01              42                                 0   
404      1.055185e+09       532000000                                 0   
1337     2.927942e+10    314159265359                                 0   
BTC      1.792718e+07        21000000                                 0   
ETH      1.076842e+08               0                                 0   

      Algorithm_536  Algorithm_Argon2d  Algorithm_BLAKE256  Algorithm_Blake  \
42                0                  0                   0                0   
404               0                  0                   0                0   
1337              0                  0                   0                0   
BTC               0                  0                   0                0   
ETH               0                  0                   0                0   

      Algorithm_Blake2S  Algorithm_Blake2b  Algorithm_C11  ...  \
42                    0                  0              0  ...   
404                   0                  0              0  ...   
1337                  0                  0              0  ...   
BTC                   0                  0              0  ...   
ETH                   0                  0              0  ...   

      ProofType_PoW/PoS  ProofType_PoW/PoS   ProofType_PoW/PoW  \
42                    1                   0                  0   
404                   1                   0                  0   
1337                  1                   0                  0   
BTC                   0                   0                  0   
ETH                   0                   0                  0   

      ProofType_PoW/nPoS  ProofType_Pos  ProofType_Proof of Authority  \
42                     0              0                             0   
404                    0              0                             0   
1337                   0              0                             0   
BTC                    0              0                             0   
ETH                    0              0                             0   

      ProofType_Proof of Trust  ProofType_TPoS  \
42                           0               0   
404                          0               0   
1337                         0               0   
BTC                          0               0   
ETH                          0               0   

      ProofType_Zero-Knowledge Proof  ProofType_dPoW/PoW  
42                                 0                   0  
404                                0                   0  
1337                               0                   0  
BTC                                0                   0  
ETH                                0                   0  

[5 rows x 100 columns]

In [13]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(X)
print(crypto_scaled[0])

[-0.11674788 -0.15286468 -0.0433555  -0.0433555  -0.0433555  -0.06137164
 -0.07523548 -0.0433555  -0.06137164 -0.06137164 -0.0433555  -0.0433555
 -0.19226279 -0.06137164 -0.09731237 -0.0433555  -0.11536024 -0.07523548
 -0.0433555  -0.0433555  -0.15176505 -0.0433555  -0.13105561 -0.0433555
 -0.0433555  -0.08695652 -0.0433555  -0.0433555  -0.0433555  -0.0433555
 -0.06137164 -0.0433555  -0.08695652 -0.08695652 -0.08695652 -0.0433555
 -0.13105561 -0.13827675 -0.13827675 -0.0433555  -0.06137164 -0.0433555
 -0.07523548 -0.1815096  -0.0433555  -0.0433555  -0.0433555  -0.07523548
 -0.15811388 -0.3145935  -0.0433555  -0.08695652 -0.07523548 -0.06137164
 -0.0433555   1.38873015 -0.0433555  -0.0433555  -0.06137164 -0.0433555
 -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555
 -0.0433555  -0.39836623 -0.0433555  -0.1815096  -0.0433555  -0.08695652
 -0.08695652 -0.10670145 -0.0433555  -0.0433555  -0.13105561 -0.0433555
 -0.0433555  -0.0433555  -0.0433555  -0.07523548 -0.4386271

### Reducing Dimensions Using PCA

In [14]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)

In [15]:
# Create a DataFrame with the principal components data
df_crypto_pca = pd.DataFrame(
    data=crypto_pca, index = coins_name.index, columns=["PC 1", "PC 2", "PC 3"]
)
df_crypto_pca.head()

PC 1      PC 2      PC 3
42   -0.352952  0.933985 -0.561562
404  -0.336362  0.933959 -0.561816
1337  2.281324  1.630911 -0.534256
BTC  -0.121612 -1.204840  0.203695
ETH  -0.128690 -1.945068  0.349864

### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [16]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using Altair
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

alt.Chart(df_elbow, title = "Elbow Curve").mark_line().encode(
    x='k',
    y='inertia'
   )

alt.Chart(...)

Running K-Means with `k = 4`

In [17]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(df_crypto_pca)

# Predict clusters
predictions = model.predict(df_crypto_pca)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
df_crypto_pca["Class"] = model.labels_
clustered_df = pd.concat([crypto_features_df, coins_name, df_crypto_pca], axis = 1)
cols = ["Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply", "PC 1", "PC 2","PC 3", "CoinName", "Class"]
clustered_df = clustered_df[cols]
clustered_df.head()

Algorithm ProofType  TotalCoinsMined TotalCoinSupply      PC 1      PC 2  \
42      Scrypt   PoW/PoS     4.199995e+01              42 -0.352952  0.933985   
404     Scrypt   PoW/PoS     1.055185e+09       532000000 -0.336362  0.933959   
1337       X13   PoW/PoS     2.927942e+10    314159265359  2.281324  1.630911   
BTC    SHA-256       PoW     1.792718e+07        21000000 -0.121612 -1.204840   
ETH     Ethash       PoW     1.076842e+08               0 -0.128690 -1.945068   

          PC 3   CoinName  Class  
42   -0.561562    42 Coin      0  
404  -0.561816    404Coin      0  
1337 -0.534256  EliteCoin      0  
BTC   0.203695    Bitcoin      1  
ETH   0.349864   Ethereum      1

### Visualizing Results

#### 2D-Scatter with Clusters

In [18]:
# Create a 2D-Scatter with the PCA data and the clusters

alt.Chart(clustered_df).mark_circle(size=60).encode(
    x='PC 1',
    y='PC 2',
    color= alt.Color('Class', scale=alt.Scale(range=['red', 'purple'])),
    tooltip=['CoinName', 'Algorithm', 'TotalCoinsMined', 'TotalCoinSupply']
).interactive()



alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [19]:
# Table with tradable cryptos
columns = clustered_df[['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']]
display(columns)

CoinName        Algorithm       ProofType TotalCoinSupply  \
42               42 Coin           Scrypt         PoW/PoS              42   
404              404Coin           Scrypt         PoW/PoS       532000000   
1337           EliteCoin              X13         PoW/PoS    314159265359   
BTC              Bitcoin          SHA-256             PoW        21000000   
ETH             Ethereum           Ethash             PoW               0   
LTC             Litecoin           Scrypt             PoW        84000000   
DASH                Dash              X11         PoW/PoS        22000000   
XMR               Monero   CryptoNight-V7             PoW               0   
ETC     Ethereum Classic           Ethash             PoW       210000000   
ZEC                ZCash         Equihash             PoW        21000000   
BTS            Bitshares          SHA-512             PoS      3600570502   
DGB             DigiByte         Multiple             PoW     21000000000   
BTCD         BitcoinDark          SHA-256         PoW/PoS        22000000   
XPY              PayCoin          SHA-256             PoS        12500000   
PRC          ProsperCoin           Scrypt             PoW        21000000   
KOBO            KoboCoin              X15         PoW/PoS       350000000   
SPR           Spreadcoin              X11             PoW        20000000   
ARG             Argentum           Scrypt             PoW        64000000   
AUR          Aurora Coin           Scrypt         PoW/PoS        16768584   
BLU             BlueCoin           Scrypt         PoW/PoS               0   
XMY           MyriadCoin         Multiple             PoW      2000000000   
MOON            MoonCoin           Scrypt             PoW    384000000000   
ZET             ZetaCoin          SHA-256             PoW       169795588   
SXC              SexCoin           Scrypt             PoW       250000000   
QTL              Quatloo           Scrypt             PoW       100000000   
ENRG          EnergyCoin           Scrypt         PoW/PoS               0   
QRK            QuarkCoin            Quark             PoW       247000000   
RIC              Riecoin          Groestl             PoW        84000000   
DGC         Digitalcoin            Scrypt             PoW        48166000   
BTB               BitBar           Scrypt         PoW/PoS          500000   
...                  ...              ...             ...             ...   
MI            XiaoMiCoin           Scrypt         PoW/PoS       400000000   
GRS          Groestlcoin          Groestl             PoW       105000000   
CPC            CapriCoin              X11         PoW/PoS       208000000   
CLUB            ClubCoin           Scrypt         PoW/PoS       160000000   
RADS              Radium              PoS             PoS         9000000   
BTA                 Bata           Scrypt             PoW         5000000   
PAK              Pakcoin           Scrypt             PoW       182000000   
CRB           Creditbit               X11  Proof of Trust        16504333   
OK                OKCash          SHA-256         PoW/PoS       105000000   
LSK                 Lisk             DPoS            DPoS       159918400   
XHI               HiCoin           Scrypt             PoS     10008835635   
XWC            WhiteCoin           Scrypt         PoW/PoS       300000000   
FSC       FriendshipCoin        NeoScrypt         PoW/PoS        60168145   
FIII                Fiii         SHA3-256            DPoC      5000000000   
J               JoinCoin         Multiple             PoW         2800000   
TRI       Triangles Coin              X13         PoW/PoS          120000   
VOLLAR            Vollar  Equihash+Scrypt             PoW      2100000000   
EOS                  EOS             DPoS            DPoS               0   
RCC        Reality Clash           Ethash             PoW        24487944   
OXY              Oxycoin             DPoS            DPoS               0   
TCH         

In [20]:
# Print the total number of tradable cryptocurrencies
print(f"Total number of tradable cryptocurrencies : {len(clustered_df.CoinName)}")

Total number of tradable cryptocurrencies : 533


#### Scatter Plot with Tradable Cryptocurrencies

In [21]:
# Scale data to create the scatter plot
columns = ["TotalCoinsMined", "TotalCoinSupply"]
clustered_scaled = StandardScaler().fit_transform(clustered_df[columns])

scaled_columns = ["TotalCoinsMined_scaled", "TotalCoinSupply_scaled"]
df_clustered_scaled = pd.DataFrame(
    data=clustered_scaled, index = clustered_df.index,
    columns=scaled_columns)
# df_clustered_scaled.head()

tradable_crypto = pd.concat([clustered_df, df_clustered_scaled], axis = 1)
tradable_crypto.head()



Algorithm ProofType  TotalCoinsMined TotalCoinSupply      PC 1      PC 2  \
42      Scrypt   PoW/PoS     4.199995e+01              42 -0.352952  0.933985   
404     Scrypt   PoW/PoS     1.055185e+09       532000000 -0.336362  0.933959   
1337       X13   PoW/PoS     2.927942e+10    314159265359  2.281324  1.630911   
BTC    SHA-256       PoW     1.792718e+07        21000000 -0.121612 -1.204840   
ETH     Ethash       PoW     1.076842e+08               0 -0.128690 -1.945068   

          PC 3   CoinName  Class  TotalCoinsMined_scaled  \
42   -0.561562    42 Coin      0               -0.116748   
404  -0.561816    404Coin      0               -0.093589   
1337 -0.534256  EliteCoin      0                0.525872   
BTC   0.203695    Bitcoin      1               -0.116354   
ETH   0.349864   Ethereum      1               -0.114384   

      TotalCoinSupply_scaled  
42                 -0.152865  
404                -0.144996  
1337                4.493764  
BTC                -0.152554  
ETH                -0.152865

In [22]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
alt.Chart(tradable_crypto).mark_circle(size=60).encode(
    x="TotalCoinsMined_scaled",
    y="TotalCoinSupply_scaled",
    color= alt.Color('Class', scale=alt.Scale(range=['red', 'blue'])),
    tooltip=['CoinName']
).interactive()


alt.Chart(...)